In [ ]:
! pip install bs4 
#in case you don't have it installed
# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz
! pip install -U gensim
import nltk
nltk.download('wordnet') 

In [3]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk
import re
from sklearn.utils import shuffle
import contractions
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
import string

C:\Users\leo\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\leo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/d

## Read Data

In [4]:
 df=pd.read_csv("amazon_reviews_us_Jewelry_v1_00.tsv",sep='\t',error_bad_lines=False)

b'Skipping line 10437: expected 15 fields, saw 22\nSkipping line 10443: expected 15 fields, saw 22\nSkipping line 19872: expected 15 fields, saw 22\nSkipping line 20055: expected 15 fields, saw 22\nSkipping line 20107: expected 15 fields, saw 22\nSkipping line 20167: expected 15 fields, saw 22\nSkipping line 53858: expected 15 fields, saw 22\n'
b'Skipping line 72173: expected 15 fields, saw 22\nSkipping line 84308: expected 15 fields, saw 22\nSkipping line 92156: expected 15 fields, saw 22\nSkipping line 97791: expected 15 fields, saw 22\nSkipping line 106812: expected 15 fields, saw 22\n'
b'Skipping line 138899: expected 15 fields, saw 22\nSkipping line 145840: expected 15 fields, saw 22\nSkipping line 192385: expected 15 fields, saw 22\nSkipping line 194126: expected 15 fields, saw 22\n'
b'Skipping line 222872: expected 15 fields, saw 22\nSkipping line 236587: expected 15 fields, saw 22\nSkipping line 245017: expected 15 fields, saw 22\n'
b'Skipping line 273895: expected 15 fields, s

In [5]:
df.dropna(inplace=True)
df[['star_rating']]=df[['star_rating']].astype(int)
df['review_body']=df[['review_headline', 'review_body']].apply(" ".join, axis=1)
df_select=df[['star_rating','review_body']]
df.groupby(['star_rating']).size()

star_rating
1     155000
2     100791
3     159644
4     270420
5    1080847
dtype: int64

## 1.Dataset Generation

In [57]:
#build a balanced dataset of 100k reviews
def prepare(df):
    df.dropna(inplace=True)
    df[['star_rating']]=df[['star_rating']].astype(int)
    df_select=df[['star_rating','review_body']]

    gp=shuffle(df_select[df_select.star_rating==1])
    gp=gp.head(20000)
    temp=gp
    for i in [2,3,4,5]:
        gp=shuffle(df_select[df_select.star_rating==i])
        gp=gp.head(20000)
        temp=pd.concat([temp,gp])
        
    return temp

,star_rating,review_body
1199322,1,Not a lip ring! To small to be a lip ring! So ...
858715,1,Design has chain attached to bangle with dangl...
215960,1,two small for my 9 years old and very difficul...
1611668,1,These earrings WERE NOT a pleasant surprise bu...
22545,1,Huge!
...,...,...
225058,5,Top quality. Great ring. It looks just as good...
1744151,5,"this is a very pretty ring, I wear it as a wed..."
693646,5,Beautiful bracelet my daughter lives it thank you
856725,5,This bracelet is very pretty and a wonderful w...


## 2.Word Embedding

## a. Pretrained Word2Vec model

In [6]:
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api

In [7]:
model_g=api.load("word2vec-google-news-300")

In [68]:
print('similarity of excellent and outstanding:'+str(model_g.similarity('excellent','outstanding')))

similarity of excellent and outstanding:0.55674857


In [69]:
print('King-Man+Woman=')
print(model_g.most_similar(positive=['woman', 'king'], negative=['man'], topn=1))

King-Man+Woman=
[('queen', 0.7118193507194519)]


## b. Train my own Word2Vec model

In [8]:
#build a balanced dataset of 100k reviews
def prepare(df):
    df.dropna(inplace=True)
    df[['star_rating']]=df[['star_rating']].astype(int)
    df_select=df[['star_rating','review_body']]

    gp=shuffle(df_select[df_select.star_rating==1])
    gp=gp.head(20000)
    temp=gp
    for i in [2,3,4,5]:
        gp=shuffle(df_select[df_select.star_rating==i])
        gp=gp.head(20000)
        temp=pd.concat([temp,gp])
        
    return temp

dataset=prepare(df)
dataset

,star_rating,review_body
1749548,1,Do not order this belly ring! What you see in ...
1036966,1,Photo is very deceptive The photo is not the r...
1172066,1,LOOKED LIKE A GUMBALL TRINKET(but not as nice)...
1749746,1,"live, love and laugh bracelet Engraving was b..."
332978,1,"One Star Does not look nice at all in person, ..."
...,...,...
837340,5,Love it This necklace took a while to receive ...
1661508,5,"Great. I was shopping for nice, shiny blue top..."
1553782,5,Like it! I bought this for my friend as a pres...
339543,5,"Five Stars Perfect size and very pretty , My ..."


In [9]:
temp=dataset
#lowercase
temp['review_body']=temp.review_body.str.lower()
#perform contractions
temp['review_body']=temp['review_body'].apply(lambda x: [contractions.fix(word) for word in x.split()])
temp['review_body']=[' '.join(map(str, l)) for l in temp['review_body']]
#remove tag
temp['review_body'] = temp['review_body'].apply(lambda cw: BeautifulSoup(cw).get_text())
temp['review_body'] = temp['review_body'].apply(lambda x: re.sub(r"http\S+", "", x))
#tokenize
temp['review_body']=temp['review_body'].apply(nltk.word_tokenize)
# remove non-alphabetical
temp['review_body'] = temp['review_body'].apply(lambda x: [re.sub('[^a-zA-Z]', '', word) for word in x])
temp['review_body']=[' '.join(map(str, l)) for l in temp['review_body']]
#tokenize
temp['review_body']=temp['review_body'].apply(nltk.word_tokenize)
temp['review_body']

1749548    [do, not, order, this, belly, ring, what, you,...
1036966    [photo, is, very, deceptive, the, photo, is, n...
1172066    [looked, like, a, gumball, trinket, but, not, ...
1749746    [live, love, and, laugh, bracelet, engraving, ...
332978     [one, star, does, not, look, nice, at, all, in...
                                 ...                        
837340     [love, it, this, necklace, took, a, while, to,...
1661508    [great, i, was, shopping, for, nice, shiny, bl...
1553782    [like, it, i, bought, this, for, my, friend, a...
339543     [five, stars, perfect, size, and, very, pretty...
851559     [big, but, beautiful, a, little, big, for, my,...
Name: review_body, Length: 100000, dtype: object

In [10]:
vec_dataset = temp['review_body'].values.tolist()

In [11]:
my_model=gensim.models.Word2Vec(vec_dataset, vector_size=300, window=11, min_count=10)

In [12]:
print('King-Man+Woman=')
print(my_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1))

King-Man+Woman=
[('differ', 0.542309045791626)]


In [13]:
print('similarity of excellent and outstanding:'+str(my_model.wv.similarity('excellent','outstanding')))

similarity of excellent and outstanding:0.7342691


<font size=4> Explanation:
In case of 'king+woman-man=queen', the pretrained model performs better. In case of 'excellent~outstanding', my model performs better. The reason behind the result might be our dataset is reviews of jewellery and therefore excellent, outstanding will be two words that appear more frequently which let our model predict the similarity better. However, queen, king may appear less in our dataset than in the google dataset, and therefore pretrained model predicts this similarity better. </font>

## 3. Simple Model

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
#build a balanced dataset of 100k reviews
def prepare(df):
    df.dropna(inplace=True)
    df[['star_rating']]=df[['star_rating']].astype(int)
    df_select=df[['star_rating','review_body']]

    gp=shuffle(df_select[df_select.star_rating==1])
    gp=gp.head(20000)
    temp=gp
    for i in [2,3,4,5]:
        gp=shuffle(df_select[df_select.star_rating==i])
        gp=gp.head(20000)
        temp=pd.concat([temp,gp])
        
    return temp

dataset=prepare(df)
dataset

,star_rating,review_body
1445861,1,"Ehh, okay ring I choose this product for a New..."
1549772,1,Gold or Plastic ??? This was to be a special g...
1174944,1,Cheap POS Ordered this necklace as a gift and ...
921182,1,very disappointed very disappointed in product...
1479498,1,Piece of junk my husband got this for me for C...
...,...,...
638727,5,Original necklace. Love it. Very beautiful. I ...
1756357,5,Love it! This Ring Holder is so much more fun ...
1745468,5,"Silver Hoop Earrings Good quality, good size e..."
1189472,5,"Beautiful with a beautiful verse Beautiful, an..."


In [15]:
temp=dataset
#lowercase
temp['review_body']=temp.review_body.str.lower()
#perform contractions
temp['review_body']=temp['review_body'].apply(lambda x: [contractions.fix(word) for word in x.split()])
temp['review_body']=[' '.join(map(str, l)) for l in temp['review_body']]
#remove tag
temp['review_body'] = temp['review_body'].apply(lambda cw: BeautifulSoup(cw).get_text())
temp['review_body'] = temp['review_body'].apply(lambda x: re.sub(r"http\S+", "", x))
#tokenize
temp['review_body']=temp['review_body'].apply(nltk.word_tokenize)
# remove non-alphabetical
temp['review_body'] = temp['review_body'].apply(lambda x: [re.sub('[^a-zA-Z]', '', word) for word in x])
temp['review_body']=[' '.join(map(str, l)) for l in temp['review_body']]
#tokenize
temp['review_body']=temp['review_body'].apply(nltk.word_tokenize)
temp['review_body']

1445861    [ehh, okay, ring, i, choose, this, product, fo...
1549772    [gold, or, plastic, this, was, to, be, a, spec...
1174944    [cheap, pos, ordered, this, necklace, as, a, g...
921182     [very, disappointed, very, disappointed, in, p...
1479498    [piece, of, junk, my, husband, got, this, for,...
                                 ...                        
638727     [original, necklace, love, it, very, beautiful...
1756357    [love, it, this, ring, holder, is, so, much, m...
1745468    [silver, hoop, earrings, good, quality, good, ...
1189472    [beautiful, with, a, beautiful, verse, beautif...
186363     [i, have, the, necklace, and, bracelet, and, l...
Name: review_body, Length: 100000, dtype: object

In [17]:
#remove stop words
stop_words=set(stopwords.words('english'))
temp['review_body']=temp['review_body'].apply(lambda x: [word for word in x if word not in stop_words])

#perform lemmatization
lemmatizer = WordNetLemmatizer()
temp['review_body'] = temp['review_body'].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])
temp['review_body']=[' '.join(map(str, l)) for l in temp['review_body']]

temp['review_body']=temp['review_body'].apply(nltk.word_tokenize)
temp['review_body']

1445861    [ehh, okay, ring, choose, product, new, year, ...
1549772    [gold, plastic, special, gift, arrived, honest...
1174944    [cheap, po, ordered, necklace, gift, totally, ...
921182     [disappointed, disappointed, product, son, bou...
1479498    [piece, junk, husband, got, christmas, let, te...
                                 ...                        
638727     [original, necklace, love, beautiful, get, ton...
1756357    [love, ring, holder, much, fun, stuffy, crysta...
1745468    [silver, hoop, earring, good, quality, good, s...
1189472    [beautiful, beautiful, verse, beautiful, beaut...
186363     [necklace, bracelet, love, earring, necklace, ...
Name: review_body, Length: 100000, dtype: object

In [18]:
my_dataset=temp['review_body'].values.tolist()

In [23]:
tfidf_dataset=[' '.join(map(str, l)) for l in temp['review_body']]

In [24]:
# TFIDF feature extraction
tfIdfVectorizer=TfidfVectorizer(use_idf=True, max_features=5000)
tfIdf = tfIdfVectorizer.fit_transform(tfidf_dataset)

In [19]:
#labels
y=temp['star_rating'].values

In [25]:
X1_train, X1_test, y1_train, y1_test = train_test_split(tfIdf, y, test_size=0.2, random_state=1)

In [26]:
#Average Word2Vec feature extraction
w2v_ls_simple=np.zeros((100000,300),dtype='float32')
for i in range(len(my_dataset)):
    vec=[]
    for j in my_dataset[i]:
        if j in model_g.key_to_index:
            xxx=model_g[j].astype('float32')
            vec.append(xxx)
        
    if not vec:
        vec=[np.zeros(model_g[my_dataset[0][0]].shape, dtype='float32')]
    a=np.mean(vec, dtype='float32', axis=0)
    w2v_ls_simple[i]=a


In [27]:
X2_train, X2_test, y2_train, y2_test = train_test_split(w2v_ls_simple, y, test_size=0.2, random_state=1)

In [28]:
#Perceptron
w2v_p = Perceptron(tol=1e-2, random_state=1)
w2v_p.fit(X2_train, y2_train)
print('Accuracy of Word2Vec perceptron:'+ str(w2v_p.score(X2_test, y2_test)))

tf_p = Perceptron(tol=1e-2, random_state=1)
tf_p.fit(X1_train, y1_train)
print('Accuracy of TFIdf perceptron:'+ str(tf_p.score(X1_test, y1_test)))

Accuracy of Word2Vec perceptron:0.4798
Accuracy of TFIdf perceptron:0.55045


In [49]:
#SVM
w2v_svm = LinearSVC()
w2v_svm.fit(X2_train, y2_train)
w2v_svm.score(X2_test,y2_test)
print('Accuracy of Word2Vec SVM:'+ str(w2v_svm.score(X2_test, y2_test)))

tf_svm = LinearSVC()
tf_svm.fit(X1_train, y1_train)
tf_svm.score(X1_test,y1_test)
print('Accuracy of TFIdf SVM:'+ str(tf_svm.score(X1_test, y1_test)))

Accuracy of Word2Vec SVM:0.5447
Accuracy of TFIdf SVM:0.6065


<font size=4> Explanation: In both cases of SVM and Perceptron, the TF-IDF features have a better accuracy than the Word2Vec feature. Also, by comparing the two simple models, SVM tends to learning the features of texts better than the Perceptron. </font>

## 4. Feedforward Neural Networks

In [31]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
import functools

## a. Average Word2Vec vectors

<font size=4> Reference: https://www.kaggle.com/code/mishra1993/pytorch-multi-layer-perceptron-mnist/notebook <font>

In [32]:
w2v_ls_avg=[]
for i in range(len(my_dataset)):
    vec=[]
    for j in my_dataset[i]:
        if j in model_g.key_to_index:
            xxx=model_g[j].astype('float32')
            vec.append(xxx)
        
    if not vec:
        vec=[np.zeros(model_g[my_dataset[0][0]].shape, dtype='float32')]
    a=np.mean(vec, dtype='float32', axis=0)
    w2v_ls_avg.append(a)

In [33]:
y=temp['star_rating'].values
y=y.astype('float32')

In [34]:
X3_train, X3_test, y3_train, y3_test = train_test_split(w2v_ls_avg, y, test_size=0.2, random_state=1)

In [35]:
# In this cell, all the labels will minus by one since in cross-entropy, the targets always begin with 0.
# Also, the training dataset will be split into validation and training
len_train = int(0.8 * len(X3_train))
X_train_avg, y_train_avg = X3_train[:len_train], []
for i in range(len_train):
    y_train_avg.append(y3_train[i] - 1) #all the labels minus by one

len_valid = len(X3_train) - len_train
X_valid_avg, y_valid_avg = X3_train[len_train:], []
for i in range(len_valid):
    y_valid_avg.append(y3_train[len_train + i] - 1) #all the labels minus by one

X_test_avg, y_test_avg = X3_test, y3_test

In [30]:
#change the label type from float64 to float32
def changetype(x):
    xx=np.array(x, dtype='float32')
    b=xx.tolist()
    return b

In [36]:
y_train_avg=changetype(y_train_avg)
y_test_avg=changetype(y_test_avg)
y_valid_avg=changetype(y_valid_avg)

In [37]:
#a customed dataset class which will be mainly used for dataloader
class Dataset(Dataset):
    
    def __init__(self, features, labels):        
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        # Convert lists to tensors
        X = torch.tensor(self.features[index])
        y = self.labels[index]
        
        return X, y

In [39]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 0}

In [81]:
# Generate training, validation and testing datasets
train_set = Dataset(X_train_avg, y_train_avg)
valid_set = Dataset(X_valid_avg, y_valid_avg)
test_set = Dataset(X_test_avg, y_test_avg)

# Generate dataloaders for the training, validation and testing datasets
train_loader = torch.utils.data.DataLoader(train_set, **params)
valid_loader = torch.utils.data.DataLoader(valid_set, **params)
test_loader = torch.utils.data.DataLoader(test_set, **params)

In [41]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [43]:
# Feedforward neural network model
class FNN(nn.Module):
    def __init__(self, input_size, hidden1, hidden2):
        super(FNN, self).__init__()
        self.input_size=input_size
        self.hidden1=hidden1
        self.hidden2=hidden2
        
        # linear layer (300 -> hidden_1)
        self.fc1 = nn.Linear(self.input_size, self.hidden1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(self.hidden1, self.hidden2)
        # linear layer (n_hidden -> 5)
        self.fc3 = nn.Linear(self.hidden2, 5)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        return x

In [82]:
# initialize the NN
fnn = FNN(300, 50, 10)
print(fnn)

FNN(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [83]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(fnn.parameters(), lr=0.15)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# number of epochs to train the model
n_epochs = 120

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    fnn.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = fnn(data.float())
        # calculate the loss
        loss = criterion(output, target.long())
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    fnn.eval() # prep model for evaluation
    for data, target in valid_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = fnn(data.float())
        # calculate the loss
        loss = criterion(output, target.long())
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(fnn.state_dict(), 'fnn.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.458558 	Validation Loss: 1.238879
Validation loss decreased (inf --> 1.238879).  Saving model ...
Epoch: 2 	Training Loss: 1.242959 	Validation Loss: 1.158720
Validation loss decreased (1.238879 --> 1.158720).  Saving model ...
Epoch: 3 	Training Loss: 1.196788 	Validation Loss: 1.122065
Validation loss decreased (1.158720 --> 1.122065).  Saving model ...
Epoch: 4 	Training Loss: 1.162667 	Validation Loss: 1.091022
Validation loss decreased (1.122065 --> 1.091022).  Saving model ...
Epoch: 5 	Training Loss: 1.143059 	Validation Loss: 1.103347
Epoch: 6 	Training Loss: 1.125532 	Validation Loss: 1.061989
Validation loss decreased (1.091022 --> 1.061989).  Saving model ...
Epoch: 7 	Training Loss: 1.109783 	Validation Loss: 1.047499
Validation loss decreased (1.061989 --> 1.047499).  Saving model ...
Epoch: 8 	Training Loss: 1.098899 	Validation Loss: 1.050907
Epoch: 9 	Training Loss: 1.088954 	Validation Loss: 1.018150
Validation loss decreased (1.047499 --> 1.

Epoch: 105 	Training Loss: 0.924154 	Validation Loss: 0.948027
Epoch: 106 	Training Loss: 0.925113 	Validation Loss: 0.965709
Epoch: 107 	Training Loss: 0.921420 	Validation Loss: 0.958744
Epoch: 108 	Training Loss: 0.924620 	Validation Loss: 0.952713
Epoch: 109 	Training Loss: 0.924152 	Validation Loss: 0.950189
Epoch: 110 	Training Loss: 0.924000 	Validation Loss: 0.957738
Epoch: 111 	Training Loss: 0.921112 	Validation Loss: 0.946453
Epoch: 112 	Training Loss: 0.924178 	Validation Loss: 0.947657
Epoch: 113 	Training Loss: 0.920981 	Validation Loss: 0.956296
Epoch: 114 	Training Loss: 0.918902 	Validation Loss: 0.961494
Epoch: 115 	Training Loss: 0.919674 	Validation Loss: 0.950827
Epoch: 116 	Training Loss: 0.919984 	Validation Loss: 0.945964
Epoch: 117 	Training Loss: 0.917030 	Validation Loss: 0.954657
Epoch: 118 	Training Loss: 0.920527 	Validation Loss: 0.950149
Epoch: 119 	Training Loss: 0.919222 	Validation Loss: 0.957745
Epoch: 120 	Training Loss: 0.918417 	Validation Loss: 0

In [68]:
#predict the test dataset with model
def predict(model, dataloader):
    y_pred = []
    y_test = []
    for i, batch in enumerate(dataloader):
        outputs = model(batch[0])
        _, predicted = torch.max(outputs.data, 1)
        y_pred.append(predicted)
        y_test.append(batch[1])
    return y_pred, y_test

In [69]:
#convert the predictations into a list
def get_pred(pred_label, test_label):
    pred=pred_label
    label=test_label
    pred = [list(torch.Tensor.numpy(t)) for t in pred]
    label = [list(torch.Tensor.numpy(t)) for t in label]
    # Convert predictions and truths from list of lists to a single list
    pred = functools.reduce(lambda a, b: a + b, pred)
    label = functools.reduce(lambda a, b: a + b, label)
    # Convert predictions by adding all labels by one
    pred = [p + 1 for p in pred]
    
    return pred, label

In [79]:
# Load model parameters from the trained model
fnn.load_state_dict(torch.load('fnn.pt'))
pred, label = predict(fnn, test_loader)
# Convert predictions and truths from list of tensors to list of lists
pred = [list(torch.Tensor.numpy(t)) for t in pred]
label = [list(torch.Tensor.numpy(t)) for t in label]
# Convert predictions and truths from list of lists to a single list
pred = functools.reduce(lambda a, b: a + b, pred)
label = functools.reduce(lambda a, b: a + b, label)
# Convert predictions by adding all labels by one
pred = [p + 1 for p in pred]

In [85]:
fnn.load_state_dict(torch.load('fnn.pt'))
pred, label = predict(fnn, test_loader)
y_pred, y_true = get_pred(pred, label)
# report accuracy
print('Accuracy of testing dataset by FNN:'+ str(accuracy_score(y_true, y_pred)))

Accuracy of testing dataset:0.5869


## B. First 10 W2V 

In [111]:
# first ten Word2Vec feature extraction
w2v_ls_ten=[]
for i in range(len(my_dataset)):
    vec=[]
    ten_words=my_dataset[i][:10]
    for j in ten_words:
        if j in model_g.key_to_index:
            xxx=model_g[j].astype('float32')
            xxx=list(xxx)
            vec.append(xxx)
            
    w2v_ls_ten.append(vec)

In [112]:
# define a function that adds padding to the reviews with length less than 10 words
def add_padding(x):
    padding = [0 for _ in range(300)]
    if len(x) < 10:
        for i in range(10 - len(x)):
            x.append(padding)
    return x

# add paddings
train_ten = [add_padding(x) for x in w2v_ls_ten]

In [113]:
train_ten = [functools.reduce(lambda a, b: a + b, x) for x in train_ten]

In [114]:
X4_train, X4_test, y4_train, y4_test = train_test_split(train_ten, y, test_size=0.2, random_state=1)

In [115]:
# split the training dataset into 0.2 validation dataset and 0.8 training
len_train = int(0.8 * len(X4_train))
X_train_ten, y_train_ten = X4_train[:len_train], []
for i in range(len_train):
    y_train_ten.append(y4_train[i] - 1) #all labels minus one

len_valid = len(X4_train) - len_train
X_valid_ten, y_valid_ten = X4_train[len_train:], []
for i in range(len_valid):
    y_valid_ten.append(y4_train[len_train + i] - 1) #all labels minus one

X_test_ten, y_test_ten = X4_test, y4_test

In [81]:
#change the dtpe into float32
def changetype(x):
    xx=np.array(x, dtype='float32')
    b=xx.tolist()
    return b

In [116]:
y_train_ten=changetype(y_train_ten)
y_test_ten=changetype(y_test_ten)
y_valid_ten=changetype(y_valid_ten)

In [117]:
# Generate training, validation and testing datasets
train_set = Dataset(X_train_ten, y_train_ten)
valid_set = Dataset(X_valid_ten, y_valid_ten)
test_set = Dataset(X_test_ten, y_test_ten)

# Generate dataloaders for the training, validation and testing datasets
train_loader = torch.utils.data.DataLoader(train_set, **params)
valid_loader = torch.utils.data.DataLoader(valid_set, **params)
test_loader = torch.utils.data.DataLoader(test_set, **params)

In [118]:
fnn2 = FNN(3000, 50, 10)
print(fnn2)

FNN(
  (fc1): Linear(in_features=3000, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [119]:
# specify cross entropy loss as loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(fnn2.parameters(), lr=0.08)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [120]:
# train the network
# number of epochs to train the model
n_epochs = 10

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    fnn2.train() # prep model for training
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = fnn2(data)
        # calculate the loss
        loss = criterion(output, target.long())
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    fnn2.eval() # prep model for evaluation
    for data, target in valid_loader:
        output = fnn2(data)
        # calculate the loss
        loss = criterion(output, target.long())
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(fnn2.state_dict(), 'fnn2.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.337230 	Validation Loss: 1.105510
Validation loss decreased (inf --> 1.105510).  Saving model ...
Epoch: 2 	Training Loss: 1.110383 	Validation Loss: 1.043747
Validation loss decreased (1.105510 --> 1.043747).  Saving model ...
Epoch: 3 	Training Loss: 1.043294 	Validation Loss: 1.008590
Validation loss decreased (1.043747 --> 1.008590).  Saving model ...
Epoch: 4 	Training Loss: 0.998328 	Validation Loss: 0.998881
Validation loss decreased (1.008590 --> 0.998881).  Saving model ...
Epoch: 5 	Training Loss: 0.960319 	Validation Loss: 1.000180
Epoch: 6 	Training Loss: 0.927776 	Validation Loss: 0.997713
Validation loss decreased (0.998881 --> 0.997713).  Saving model ...
Epoch: 7 	Training Loss: 0.899605 	Validation Loss: 1.016796
Epoch: 8 	Training Loss: 0.870537 	Validation Loss: 1.034686
Epoch: 9 	Training Loss: 0.845514 	Validation Loss: 1.032366
Epoch: 10 	Training Loss: 0.824275 	Validation Loss: 1.056447


In [121]:
# Load model parameters from the trained model
fnn2.load_state_dict(torch.load('fnn2.pt'))
pred, label = predict(fnn2, test_loader)
y_pred2, y_true2 = get_pred(pred, label)
# report accuracy, precision, recall, and f1-score on the testing dataset
print('Accuracy of testing dataset by FNN:' + str(accuracy_score(y_true2, y_pred2)))

Accuracy of testing dataset by FNN:0.57085


<font size=4> Explanation: 
 Firstly, in Word2Vec cases, the Feedforward Neural Network performs better than the Perceptron model. In case of 'Average Word2Vec', the FNN results a better accuracy than the Perceptron one. In case of 'First ten Word2Vec', the FNN model reach a higher accuracy than the Perceptron one.  However, SVM model still betters two neural network models.
    
    
Secondly, TF-IDF features has a better accuracy than the Word2Vec features though different models are applied.
</font>

## 5. Recurrent Neural Networks

Reference: https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

## a. Simple RNN

In [50]:
w2v_ls_twty=[]
for i in range(len(my_dataset)):
    #x=np.zeros(model[my_dataset[0][0]].shape)
    vec=[]
    twty_words=my_dataset[i][:20]
    for j in twty_words:
        if j in model_g.key_to_index:
            xxx=model_g[j].astype('float32')
            xxx=list(xxx)
            vec.append(xxx)
            
    w2v_ls_twty.append(vec)

In [51]:
def add_padding(x):
    padding = [0 for _ in range(300)]
    if len(x) < 20:
        for i in range(20 - len(x)):
            x.append(padding)
    return x

# add paddings
train_twty = [add_padding(x) for x in w2v_ls_twty]

In [52]:
y=temp['star_rating'].values
y=y.astype('float32')
y

array([1., 1., 1., ..., 5., 5., 5.], dtype=float32)

In [98]:
X5_train, X5_test, y5_train, y5_test = train_test_split(train_twty, y, test_size=0.2, random_state=1)

In [99]:
len_train = int(0.8 * len(X5_train))
X_train_twty, y_train_twty = X5_train[:len_train], []
for i in range(len_train):
    y_train_twty.append(y5_train[i] - 1) # Convert all labels by minus one

len_valid = len(X5_train) - len_train
X_valid_twty, y_valid_twty = X5_train[len_train:], []
for i in range(len_valid):
    y_valid_twty.append(y5_train[len_train + i] - 1)

X_test_twty, y_test_twty = X5_test, y5_test

In [101]:
y_train_twty=changetype(y_train_twty)
y_test_twty=changetype(y_test_twty)
y_valid_twty=changetype(y_valid_twty)

In [102]:
# Generate training, validation and testing datasets
train_set = Dataset(X_train_twty, y_train_twty)
valid_set = Dataset(X_valid_twty, y_valid_twty)
test_set = Dataset(X_test_twty, y_test_twty)

# Generate dataloaders for the training, validation and testing datasets
train_loader = torch.utils.data.DataLoader(train_set, **params)
valid_loader = torch.utils.data.DataLoader(valid_set, **params)
test_loader = torch.utils.data.DataLoader(test_set, **params)

In [103]:
# Recurrent neural network model
class RNN(nn.Module):
    def __init__(self, input_size=300, hidden_size=20, output_size=5):
        super(RNN, self).__init__()
        self.input_size = input_size 
        self.output = output_size
        self.hidden1 = hidden_size
        
        # linear layer (input --> hidden)
        self.i2h = nn.Linear(self.input_size + self.hidden1, self.hidden1)
        # linear layer (hidden --> output)
        self.i2o = nn.Linear(self.input_size + self.hidden1, self.output)
    
    def forward(self, wordVec):
        batch_size = wordVec.shape[0]
        hidden = torch.zeros(batch_size, self.hidden1)
        combined = torch.cat((wordVec[:,0,], hidden), 1)
        for i in range(1, 20):
            hidden = self.i2h(combined)
            combined = torch.cat((wordVec[:,i,], hidden), 1)
        output = self.i2o(combined)
       
        return output

In [104]:
rnn = RNN()
print(rnn)

RNN(
  (i2h): Linear(in_features=320, out_features=20, bias=True)
  (i2o): Linear(in_features=320, out_features=5, bias=True)
)


In [106]:
# specify cross entropy loss as loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.008)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [107]:
# train the network

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity
n_epochs = 50

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    rnn.train() # prep model for training
    for data, target in train_loader:
        # transfer to GPU
        #data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model

        output = rnn(data)
        # calculate the loss
        loss = criterion(output, target.long())
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    rnn.eval() # prep model for evaluation
    for data, target in valid_loader:
        output = rnn(data)
        # calculate the loss
        loss = criterion(output, target.long())
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(rnn.state_dict(), 'rnn.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.609447 	Validation Loss: 1.608321
Validation loss decreased (inf --> 1.608321).  Saving model ...
Epoch: 2 	Training Loss: 1.607923 	Validation Loss: 1.607267
Validation loss decreased (1.608321 --> 1.607267).  Saving model ...
Epoch: 3 	Training Loss: 1.606779 	Validation Loss: 1.606428
Validation loss decreased (1.607267 --> 1.606428).  Saving model ...
Epoch: 4 	Training Loss: 1.605896 	Validation Loss: 1.605782
Validation loss decreased (1.606428 --> 1.605782).  Saving model ...
Epoch: 5 	Training Loss: 1.605138 	Validation Loss: 1.605282
Validation loss decreased (1.605782 --> 1.605282).  Saving model ...
Epoch: 6 	Training Loss: 1.604504 	Validation Loss: 1.604854
Validation loss decreased (1.605282 --> 1.604854).  Saving model ...
Epoch: 7 	Training Loss: 1.603940 	Validation Loss: 1.604416
Validation loss decreased (1.604854 --> 1.604416).  Saving model ...
Epoch: 8 	Training Loss: 1.603428 	Validation Loss: 1.604082
Validation loss decreased (1.60441

In [109]:
# Load model parameters from the trained model 
rnn.load_state_dict(torch.load('rnn.pt'))
pred, label = predict(rnn, test_loader)
y_pred3, y_true3 = get_pred(pred, label)
# report accuracy, precision, recall, and f1-score on the testing dataset
print('Accuracy of testing dataset by RNN:' + str(accuracy_score(y_true3, y_pred3)))

Accuracy of testing dataset by RNN:0.52465


<font size=4> Explanation: In case of 'first N Word2Vec', FNN performs better than RNN with respect to the accuracy. Since the FNN only take the first 10 word which in simple analysis, it is more likely to contain less feature representations. Therefore, a conclusion can be drawn prematurely that FNN did better in first N Word2Vec cases.

Moreover, in case of 'averge Word2Vec', the RNN in this case still does not perform better than the FNN one.
</font>

## Gated Recurrent Unit Cell

Reference:https://web.stanford.edu/~jurafsky/slp3/ed3book.pdf

In [64]:
# Gated Recurrent Unit cell
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # linear layer for reset gate
        self.fc1 = nn.Linear(self.input_size + self.hidden_size, self.hidden_size)
        # linear layer for update gate
        self.fc2 = nn.Linear(self.input_size + self.hidden_size, self.hidden_size)
        # linear layer for intermediate hidden state
        self.fc3 = nn.Linear(self.input_size + self.hidden_size, self.hidden_size)
        # linear layer for output
        self.fc4 = nn.Linear(self.input_size + self.hidden_size, self.output_size)
    
    def forward(self, wordVec):
       
        batch_size = wordVec.shape[0]
        hidden = torch.zeros(batch_size, self.hidden_size)
        combined = torch.cat((wordVec[:,0,], hidden), 1)
        sigmoid, tanh = nn.Sigmoid(), nn.Tanh()
        for i in range(1, 20):
            # reset gate
            reset = self.fc1(combined)
            reset = sigmoid(reset)
            # update gate
            update = self.fc2(combined)
            update = sigmoid(update)
            # previous hidden state passes through reset gate
            hidden2 = torch.mul(reset, hidden)
            # intermediate hidden state
            combined2 = torch.cat((wordVec[:,0,], hidden2), 1)
            hidden_inter = self.fc3(combined2)
            hidden_inter = tanh(hidden_inter)
            # hidden state
            hidden = torch.mul(1-update, hidden) + torch.mul(update, hidden_inter)
            combined = torch.cat((wordVec[:,i,], hidden), 1)
        output = self.fc4(combined)
       
        return output

In [65]:
# initialize the GRU
gru = GRU(300, 20, 5)
print(gru)

GRU(
  (fc1): Linear(in_features=320, out_features=20, bias=True)
  (fc2): Linear(in_features=320, out_features=20, bias=True)
  (fc3): Linear(in_features=320, out_features=20, bias=True)
  (fc4): Linear(in_features=320, out_features=5, bias=True)
)


In [66]:
# specify cross entropy loss as loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(gru.parameters(), lr=0.005)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [67]:
# train the network

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

n_epochs = 50

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    gru.train() # prep model for training
    for data, target in train_loader:
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = gru(data)
        # calculate the loss
        loss = criterion(output, target.long())
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    gru.eval() # prep model for evaluation
    for data, target in valid_loader:
        output = gru(data)
        # calculate the loss
        loss = criterion(output, target.long())
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(gru.state_dict(), 'gru.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.608309 	Validation Loss: 1.604893
Validation loss decreased (inf --> 1.604893).  Saving model ...
Epoch: 2 	Training Loss: 1.601456 	Validation Loss: 1.595862
Validation loss decreased (1.604893 --> 1.595862).  Saving model ...
Epoch: 3 	Training Loss: 1.586856 	Validation Loss: 1.573489
Validation loss decreased (1.595862 --> 1.573489).  Saving model ...
Epoch: 4 	Training Loss: 1.552391 	Validation Loss: 1.524219
Validation loss decreased (1.573489 --> 1.524219).  Saving model ...
Epoch: 5 	Training Loss: 1.490596 	Validation Loss: 1.452898
Validation loss decreased (1.524219 --> 1.452898).  Saving model ...
Epoch: 6 	Training Loss: 1.424194 	Validation Loss: 1.395523
Validation loss decreased (1.452898 --> 1.395523).  Saving model ...
Epoch: 7 	Training Loss: 1.378603 	Validation Loss: 1.358755
Validation loss decreased (1.395523 --> 1.358755).  Saving model ...
Epoch: 8 	Training Loss: 1.347703 	Validation Loss: 1.331320
Validation loss decreased (1.35875

In [94]:
# Load model parameters from the trained model
gru.load_state_dict(torch.load('gru.pt'))
pred, label = predict(gru, test_loader)
y_pred4, y_true4 = get_pred(pred, label)
# report accuracy, precision, recall, and f1-score on the testing dataset
print('Accuracy of testing dataset by GRU:' + str(accuracy_score(y_true4, y_pred4)))

Accuracy of testing dataset by GRU:0.52135


<font size=4> Explanation: In the same case of 'first 20 Word2Vec', RNN performs slightly better than GRU with respect to the accuracy. However, in some time, GRU also performs better. Thus, a conclusion can be drawn that two models performs equally well with first 20 Word2Vec features.  </font>